<a href="https://colab.research.google.com/github/r73psv/-batman/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22ns_panova_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import torch
import torchvision
from torch.autograd import Variable
import torchvision.transforms as transforms

In [3]:
#  Загрузка набора CIFAR10
# преобразовываем изображение в тензоры, затем нормализуем
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170498071/170498071 [00:03<00:00, 48842197.64it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
#задаем устройство на котором будет тенироваться сеть - видеопроцессор, если нет то процессор
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [5]:
#создаем свёрточную НС (3 свёрточных слоя и 3 линейных)
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    # реализация метода forward в классе Net
    def forward(self, x):
        x = self.pool(F.logsigmoid(self.conv1(x)))
        x = self.pool(F.logsigmoid(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.logsigmoid(self.fc1(x))
        x = F.logsigmoid(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
net.to(device)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [20]:
#criterion = nn.KLDivLoss()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(10):
    running_loss = 0.0
    running_acc = 0.0
    for i,data in enumerate(trainloader,1):
        img,label = data
        img = Variable(img)
        label = Variable(label)
        out = net(img)
        #print(out)
        loss = criterion(out,label) #loss
        running_loss += loss.item() * label.size(0)
        #total loss
        _,pred = torch.max(out,1)
        num_correct = (pred == label).sum()
        #accuracy = (pred == label).float().mean()
        running_acc += num_correct.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('Train{} epoch, Loss: {:.6f},Acc: {:.6f}'.format(epoch+1,running_loss / (len(trainset)),running_acc / (len(trainset))))

Train1 epoch, Loss: 1.135782,Acc: 0.603280
Train2 epoch, Loss: 0.985624,Acc: 0.648420
Train3 epoch, Loss: 1.035781,Acc: 0.634840
Train4 epoch, Loss: 0.946948,Acc: 0.659540
Train5 epoch, Loss: 0.937377,Acc: 0.664500
Train6 epoch, Loss: 0.941703,Acc: 0.663420
Train7 epoch, Loss: 0.960599,Acc: 0.657460
Train8 epoch, Loss: 0.948322,Acc: 0.665900
Train9 epoch, Loss: 0.908444,Acc: 0.673660
Train10 epoch, Loss: 0.932891,Acc: 0.668020


In [21]:
net.eval()
eval_loss = 0
eval_acc = 0
for data in testloader:
    img,label = data
    #img = Variable(img,volatile=True)
    out = net(img)
    loss = criterion(out,label)
    eval_loss += loss.item() * label.size(0)    #total loss
    _,pred = torch.max(out,1)
    num_correct = (pred == label).sum()
    eval_acc += num_correct.item()

print('Test Loss:{:.6f},Acc: {:.6f}'
      .format(eval_loss/ (len(testset)),eval_acc * 1.0/(len(testset))))

Test Loss:1.370377,Acc: 0.568400


In [12]:
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

#criterion = nn.KLDivLoss()
criterion = nn.CrossEntropyLoss() # функция потерь
optimizer = optim.RMSprop(net.parameters(), lr=0.001, momentum=0.9) # функция оптимизации

for epoch in range(4):  # устанавливает количество циклов обучения
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):

        inputs, labels = data
        # обнуляем оптимизатор
        optimizer.zero_grad()
        # прямой проход
        outputs = net(inputs)
        out=inputs
        # Расчитываем функцию потерь
        loss = criterion(outputs, labels)
        #loss = criterion(inputs,out)
        # обратный проход
        loss.backward()
         # Шаг градиентного спуска (изменение параметров НС)
        optimizer.step()
        # печать статистики
        running_loss += loss.item()
        if i % 2000 == 1999:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
    print('Finished Training')

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Finished Training
Finished Training
Finished Training
Finished Training


In [22]:
# сохраняем модель
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [23]:

dataiter = iter(testloader)
images, labels = next(dataiter)

In [24]:
#загрузка сохраненной модели
net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [25]:
# Выходные данные представляют собой вероятности для 10 классов. Чем выше вероятность, тем больше сеть считает,
# что изображение относится к определенному классу.
outputs = net(images)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(4)))

Predicted:  ship  ship  car   plane


In [26]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        # вычисление выходных данных, прогоняя изображения по сети
        outputs = net(images)
        # класс с самой высокой вероятностью  выбираем в качестве прогноза
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 56 %


In [27]:
# подсчет прогнозов для каждого класса
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # собираем прогнозы длакаждого класа
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# печать accuracy для каждого класса
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: plane is 67.1 %
Accuracy for class: car   is 62.8 %
Accuracy for class: bird  is 39.4 %
Accuracy for class: cat   is 30.2 %
Accuracy for class: deer  is 52.9 %
Accuracy for class: dog   is 43.2 %
Accuracy for class: frog  is 72.6 %
Accuracy for class: horse is 68.6 %
Accuracy for class: ship  is 68.9 %
Accuracy for class: truck is 62.7 %
